In [39]:
import pandas as pd
import numpy as np
import glob
import re
import statsmodels.api as sm
import datetime

class data_manager():
    def __init__(self):
#         self.down_paths = glob.glob('../data/原始下載資料/*.[csvxl]*')
#         self.genr = pd.read_excel(glob.glob('../data/data_genr*.xlsx')[0])
        self.convert = pd.read_excel('../data/對照清單.xlsx',engine='openpyxl')
        self.varList =  pd.read_excel('../data/varList.xlsx',engine='openpyxl')
        self.q = pd.read_excel(glob.glob('../data/data_q*.xlsx')[0],engine='openpyxl')
        self.m = pd.read_excel(glob.glob('../data/data_m*.xlsx')[0],engine='openpyxl')
#         self.y = pd.read_excel(glob.glob('../data/data_y*.xlsx')[0])
    
    def set_index_date(self,df):
        if df.index.name != 'date':
            df.set_index(['date'],inplace=True)
    def reset_index_date(self,df):
        if df.index.name == 'date':
            df.reset_index(inplace=True)
    def join_by_date(self,df1,df2):
        self.set_index_date(df1)
        self.set_index_date(df2)
        new_set = pd.concat([df1,df2],axis='columns')
        self.reset_index_date(new_set)
        return new_set

    def reset_df_date_type(self,df,freq='q'):
        d.reset_index_date(df)
        if freq == 'q':
            df['date'] = df['date'].apply(lambda x: x.strftime('%F0%q') )
        return df
    
    def create_varList(self):
        """ 收集原始下載資料的代碼、名稱、單位 """
        dires = pd.DataFrame()
        paths = self.down_paths
        for path in paths:
            df = self.read(path)
            dire = df.iloc[:2]
            eles = []
            for name in dire.columns:
                if name not in {'date','DATE','Unnamed: 0'}:
                    eles.append(name)
            dire2 = dire[eles]
            dires = pd.concat([dires,dire2],axis='columns')
        diresT = dires.T
        diresT.columns= ['名稱','單位']
        diresT.index.name = '檢索代碼'
        diresT.reset_index(inplace=True)
        def freq(x):
            if '.' in x:
                return x.split('.')[-1]
            else:
                return np.nan
        diresT['頻率'] = diresT['檢索代碼'].apply(lambda x:freq(x))
        return diresT
    
    def read(self,path):
        """ 原始下載資料可能有xls,xlsx,csv """
        if path.split('.')[-1] in 'xlsx':
            df = pd.read_excel(path)
        elif path.split('.')[-1] == 'csv':
            df = pd.read_csv(path)
        else:
            print(path)
            return 
        return df
    def down_names(self):
        """ 原始下載資料中所有檢索代碼 """
        paths = self.down_paths
        names = []
        for path in paths:
            df = self.read(path)
            names.extend(df.columns)
        names = [x for x  in {x for x in names}]
        for i in ['Unnamed: 0','date','DATE']:
            if i in names:
                names.remove(i)
        return names
    
    def losts_varname(self):
        """ varList.xlsx少放入的檢索代碼 """
        d_name = self.down_names()
        varList = self.varList
        if varList.index.name =='檢索代碼':
            varList.reset_index('檢索代碼',inplace=True)
        losts = []
        for i in d_name:
            if i not in {x for x in varList['檢索代碼']}:
                print(i,end=',')
                losts.append(i)
        if len(losts) == 0:
            print('varList無缺漏')
            return
        return losts
    def losts_df(self):
        """ varList.xlsx少放入的df(多變數) """
        losts = self.losts_varname()
        if not losts:
            return
        diresT = self.create_varList()
        news = diresT.set_index('檢索代碼').loc[losts]
        return news
    def varList_update(self,save=None):
        news = d.losts_df()
        if type(news) != pd.core.frame.DataFrame:
            return
        varList = d.varList
        varList.set_index('檢索代碼',inplace=True)
        varList = pd.concat([news,varList],axis='index')
        if save:
            varList.to_excel('../data/varList.xlsx',index=True)
        return varList
    
    def data_names(self):
        mqy =[]
        for cols in (self.m.columns,self.q.columns,self.y.columns,self.genr.columns):
            mqy.extend(cols)
        return mqy
    def miss_data_names(self):
        missDataNames=[]
        qmy = {x for x in self.data_names()}
        dows = self.down_names()
        for dow in dows:
            if dow not in qmy:
                missDataNames.append(dow)
        return missDataNames
    
#     def down_variable(self,name,freq):
#         """ 讀取下載的變數並命名第一行為'date' """
#         paths = self.down_paths
#         for path in paths:
#             df = self.read(path)
#             if name in df.columns:
#                 break
#         if 'are' in path:
#             df = df.iloc[2:].copy()
#         df.rename(columns = {df.columns[0]:'date'},inplace=True)
#         downVari = df[[df.columns[0],name]].copy()
#         downVari['date'] = downVari['date'].astype('int64')
#         return downVari
#     def update_down_variable(self,name,freq):
#         """ 確認後再儲存 """
#         if freq == 'q':
#             data_set = self.q
#         elif freq == 'm':
#             data_set = self.m
#         downVari = self.down_variable(name,freq)
#         self.set_index_date(data_set)
#         self.set_index_date(downVari)
#         new_set = pd.concat([downVari,data_set],axis='columns')
#         selecter().reset_index_date(new_set)
#         return new_set

    def down_df(self,path):
        """ 讀取下載的檔案，更換成統一的date格式: yyyymm """
        df = d.read(path)
        if 'are_' in path:
            df = df.iloc[2:].copy()
        df.rename(columns = {df.columns[0]:'date'},inplace=True)
        if 'are_' in path:
            df['date'] = df['date'].apply(lambda x: pd.Period(x,'m').strftime('%F%m'))
        else:
            def turn(x):
                qx = pd.Period(freq='q',year= x.year,month=x.month)
                yyyymm = str(qx).replace('Q','0') # qx.strftime('%Y%Q')
                return yyyymm
            df['date'] = df['date'].apply(lambda x: turn(x))
        df['date'] = df['date'].astype('int64')
        return df
    def join_down_df(self,paths):
        """ 合併多個下載檔案"""
        data = pd.DataFrame(columns=['date'])
        for path in paths:
            df = self.down_df(path)
            data = self.join_by_date(data,df)
        return data
    
    def update_down_df(self,path,freq):
        """ 將一個或多個下載檔案合併至總資料"""
        if type(path) != str:
            df = self.join_down_df(paths)
        else:
            df = self.down_df(path)
        if freq == 'q':
            data_set = self.q
        elif freq == 'm':
            data_set = self.m
        return self.join_by_date(df,data_set)
    
    def plus_lag_df(self,df,lags,col=1):
        df.dropna(inplace=True)
        ser = [np.nan]*lags + [ x for x in df.iloc[:,col]][:-lags]
        df['{}(-{})'.format(df.columns[col],lags)] = ser
        return df
    def plus_dif_df(self,df,lags,name,col=1,):
        df = self.plus_lag_df(df,lags,col)
        df[name] = df.iloc[:,col] - df.iloc[:,col+1]
        return df
    
    def update_genr(self,df,name,freq='q'):
        #d.reset_index_date(df)
        #df = df[['date','(IRC-IRCUS)']].copy()
        df = df[['date',name]].copy()
        #df = self.reset_df_date_type(df,freq)
        df['date']= df['date'].astype('int64')
        print('原data_genr.shape為',d.genr.shape)
        data = self.join_by_date(df,d.genr)

        day= datetime.datetime.today().date().strftime('%m%d')
        print('data.to_excel(\'../data/data_genr{}.xlsx\',index=False)'.format(day))
        return data
    

d =data_manager()   

# 新下載資料時

## 1.手動填寫對照清單
    for: 輸入建甫模型的變數名稱後可轉成指定的aremos名稱

### 2.更新 varList.xlsx
    此為所有變數的列表

In [81]:
# d.varList_update()
# d.varList_update(save=True)

WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
S$C.m,WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero


,名稱,單位,頻率,來源
檢索代碼,,,,
S$C.m,製造業銷售價值 - 製造業 (新臺幣百萬元),新臺幣百萬元,m,NaN
CD3M,3-Month Certificate of Deposit: Secondary Mark...,%(%),m,fred
Q924L99b.q,"China, P_R_: Mainland_National Accounts, Expen...",Domestic Currency,q,NaN
Q924L99bip.q,"China, P_R_: Mainland_National Accounts, Expen...",Index,q,NaN
MR@CH.m,準備貨幣 (期底) - 回溯調整準備金後之年增率 (％),%(%),m,NaN
...,...,...,...,...
RMIBON.a,加權平均金融業隔夜拆款利率 (年息百分比率),年息百分比率,a,NaN
JS.a,發行量加權股價指數 (基期: 民國 55 年=100),基期: 民國 55 年=100 (base: 1966=100),a,NaN
JSC.a,製造業銷售量指數 - 製造業類指數 (基期:民國 105 年 = 100),基期:民國 105 年 = 100,a,NaN


## 3. 更新月或季的總資料
    兩種途徑:讀取出單一變數序列或是整個excel檔

In [96]:
# print(d.miss_data_names())
# name = d.miss_data_names()[0]
# df = d.down_variable(name,'m')
# df.columns

WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
['S$C.m']
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero


Index(['date', 'S$C.m'], dtype='object')

### 3.2 讀取excel檔

In [15]:
# paths = [d.down_paths[4]] + d.down_paths[-4:-2]
# paths

['../data/原始下載資料\\are_m1229.xls',
 '../data/原始下載資料\\fred_m_IQ.xls',
 '../data/原始下載資料\\fred_m_POILBREUSDM.xls']

In [18]:
# # date格式不統一的問題
# df1 = pd.read_excel(paths[1])
# df1.head()

,date,IQ
0,1988-12-01,93.8
1,1989-01-01,94.9
2,1989-02-01,94.6
3,1989-03-01,95.2
4,1989-04-01,95.0


In [41]:
# data = d.update_down_df(paths,'m')
# data

WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero


,date,N.m,IQ,POILBREUSDM,S$C.m,CD3M,MR@CH.m,MR.m,DALB.m,LDALB.m,...,AFI@L&D.m,RMIBON@H.m,RMIBON@L.m,RMCBC@RDISC@.m,STOCKPRICE.m,CPI@F.m,STOCKTRADE.m,RU.m,JSC.m,STOCKPRICE@.m
0,195908,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,195909,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,195910,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,195911,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,195912,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
732,202008,23424.00,122.1,44.257143,988569.69,NaN,10.66,4550042.0,37740351.0,28335612.0,...,31746235.0,0.24,0.08,1.13,12743.1,102.93,4588770.0,3.99,NaN,12743.0
733,202009,23418.00,122.8,41.085455,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
734,202010,23414.00,123.1,40.470000,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
735,202011,NaN,123.8,43.224286,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [42]:
# data.to_excel('../data/data_m1230.xlsx',index=False)

## 儲存運算出來的變數

In [33]:
# df= s.smpl_all(['IRC','IRCUS'])
# df['(IRC-IRCUS)'] = df.iloc[:,0] - df.iloc[:,1]
# d.reset_index_date(df)
# df = df[['date','(IRC-IRCUS)']].copy()
# data = d.update_genr(df,'(IRC-IRCUS)')

data.to_excel('../data/data_genr1119.xlsx',index=False)


## other

In [116]:
# smpl = s.smpl_all(['SALES','GDP'])
# smpl['SALES/GDP'] = smpl.iloc[:,0] / smpl.iloc[:,1]
# d.reset_index_date(smpl)
# df = smpl[['date','SALES/GDP']].copy()
# df = d.reset_df_date_type(df,'q')
# df['date']= df['date'].astype('int64')
# data = d.join_by_date(df,d.genr)
# #data.to_excel('../data/data_genr1118.xlsx',index=False)
# data

,date,SALES/GDP,D(IRC-@PCHY(CPIZF)),D(LOG(GDP))
0,196101,NaN,NaN,NaN
1,196102,NaN,NaN,-0.015799
2,196103,NaN,NaN,-0.063100
3,196104,NaN,NaN,0.214707
4,196201,NaN,NaN,-0.070554
...,...,...,...,...
233,201902,0.217397,1.045306,0.007159
234,201903,0.211374,1.045195,0.036885
235,201904,0.214741,1.044518,0.032282
236,202001,0.215661,0.966968,-0.063271


In [29]:
# df = d.genr
# df.rename(columns={df.columns[1]:'D(LOG(GDP))'},inplace=True)
# df.to_excel('../data/data_genr.xlsx',index=False)

In [22]:
# path = self.down_paths[0]
# df = d.update_down_df(path,'q')
# df.to_excel('../data/data_q1116.xlsx',index=False)

In [361]:
# missDataNames = d.miss_data_names()
# missDataNames

['Q924L99bip.q', 'Q924L99b.q', 'CD3M']

In [520]:
# name = missDataNames[0]
# df = d.update_down_variable(name,'q')

In [235]:
# d.varList_update()

varList無缺漏


In [223]:
# d.varList_update(save=True)

CD3M,

,名稱,單位,頻率
檢索代碼,,,
CD3M,3.86,3.87,NaN
Q924L99b.q,"China, P_R_: Mainland_National Accounts, Expen...",Domestic Currency,q
Q924L99bip.q,"China, P_R_: Mainland_National Accounts, Expen...",Index,q
MR@CH.m,準備貨幣 (期底) - 回溯調整準備金後之年增率 (％),%(%),m
MR.m,準備貨幣 (月底數) (新台幣百萬元),新台幣百萬元 (NT$ million),m
...,...,...,...
RMIBON.a,加權平均金融業隔夜拆款利率 (年息百分比率),年息百分比率,a
JS.a,發行量加權股價指數 (基期: 民國 55 年=100),基期: 民國 55 年=100 (base: 1966=100),a
JSC.a,製造業銷售量指數 - 製造業類指數 (基期:民國 105 年 = 100),基期:民國 105 年 = 100,a
